# Travel Comment Sentiment Classification
### YouTube Travel Buzz NLP Pipeline – Module 2

Multi-class sentiment classification model (Negative / Neutral / Positive)
applied to travel-related YouTube comments.


### 0. Execution Note

This notebook contains **actual training, evaluation, and inference outputs**
generated during the project period.

Due to the use of private local paths and time-bound compute environments:
- File paths were anonymized
- Authentication-related steps were removed
- The notebook is **not intended for re-execution**

All model logic, metrics, and outputs remain unchanged.


### 1. Environment Setup

Required libraries for sentiment model training and inference.


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch
from sklearn.metrics import accuracy_score, f1_score


### 2. Training Data Loading

- Input: CSV file containing `comment` and `label`
- Labels: 0 (Negative), 1 (Neutral), 2 (Positive)

In [ ]:
file_path = "data/travel_comment_sentiment_train.csv"  # anonymized path

# NOTE:
# Original data was loaded from a local private path.
# Path anonymized for public GitHub release.
df = pd.read_csv(file_path)


### 3. Dataset Construction

Convert pandas DataFrame to Hugging Face Dataset and split into train/test.

In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)


### 4. Model & Tokenizer Initialization

Base model: `monologg/koelectra-base-discriminator`


In [ ]:
model_name = "monologg/koelectra-base-discriminator"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=3
)


### 5. Tokenization

In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["comment"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("sentiment", "labels")


### 6. Evaluation Metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "f1_micro": f1_score(labels, preds, average="micro"),
    }


### 7. Training Configuration

In [ ]:
training_args = TrainingArguments(
    output_dir="./sentiment_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    logging_strategy="steps",
    logging_steps=50,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    seed=42,
    logging_dir="./sentiment_logs",
)


### 8. Model Training

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


### 9. Model Saving

In [4]:
# Model was saved locally during the project period.
# Path omitted for public release.

save_path = "./models/travel_comment_classifier_sentiment_3class"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f"모델 저장 완료: {save_path}")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 960/960 [00:00<00:00, 18954.72 examples/s]
C:\Users\rhksd\AppData\Local\Temp\ipykernel_15204\2031024620.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro
1,0.227900,0.174383,0.940625,0.939536,0.940625
2,0.073200,0.141200,0.956250,0.955907,0.956250
3,0.067900,0.119921,0.962500,0.962232,0.962500
4,0.045900,0.127640,0.962500,0.962210,0.962500
5,0.027000,0.126816,0.963542,0.963291,0.963542


모델 저장 완료: C:\Users\rhksd\Desktop\Data\항공\travel_comment_classifier_sentiment_3class


### 10. Sentiment Comment Inference Setup

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
from pathlib import Path

model_path = Path("model/travel_comment_sentiment_classifier")  # placeholder

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

def predict_comment(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        
    return pred


### 11. Batch Sentiment Inference on YouTube Comments

Apply the classifier to real YouTube comments and analyze confidence.

In [ ]:
from pathlib import Path

file_path = Path("data/youtube_travel_comments.csv")  # anonymized
df = pd.read_csv(file_path)

df["pred_label"] = df["comment"].apply(predict_comment)


### 12. Sentiment Distribution Summary

In [5]:
label_map = {0: "부정", 1: "중립", 2: "긍정"}

summary = (
    df["pred_label"]
    .value_counts()
    .sort_index()
    .rename(index=label_map)
)

total = len(df)

print("\n🔎 감정 분포 요약:")
for label, count in summary.items():
    pct = count / total * 100
    print(f"  {label: <4}: {count}개 ({pct:.2f}%)")

print(f"\n✅ 총 댓글 수: {total}개")
print(f"📁 결과 저장 완료: {output_path}")



🔎 감정 분포 요약:
  부정  : 61개 (10.50%)
  중립  : 56개 (9.64%)
  긍정  : 464개 (79.86%)

✅ 총 댓글 수: 581개
📁 결과 저장 완료: C:\Users\rhksd\Desktop\Data\항공\drive-download-20250604T053503Z-1-001\phuquoc_comment_data_2023-01_labeled.csv


### 13. Interpretation of Inference Results

The inference results show that the sentiment classifier produces a stable and interpretable distribution across negative, neutral, and positive classes on real-world travel-related comments.

Key observations from the output include:

- Positive sentiment accounts for the majority of comments, which is consistent with the aspirational and experiential nature of travel content.
- Negative sentiment appears in smaller proportions, suggesting that dissatisfaction is present but not dominant.
- Neutral sentiment represents a meaningful share of comments, reflecting information-seeking behavior such as questions, factual statements, and planning-related inquiries.

This confirms that the three-class sentiment formulation avoids forcing ambiguous comments into polar categories and better captures realistic user discourse.

The resulting sentiment labels are used as aggregated signals in downstream analysis rather than as standalone judgments on individual comments.